In [ ]:
# You don't need these two lines if you have pip installed topologicpy
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

## Import the needed classes

In [ ]:
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Topology import Topology
from topologicpy.CellComplex import CellComplex
from topologicpy.Graph import Graph
from topologicpy.Kuzu import Kuzu
from topologicpy.Dictionary import Dictionary
from topologicpy.Helper import Helper

## Check Software Versions

In [ ]:
import sys
print("This workflow was designed and verified to work with python 3.11 and TopologicPy 0.8.58.\n")
py_version = sys.version.split()[0]
print("Python version:", py_version)

print("TopologicPy Version:", Helper.Version(check=False))
print(" ")
print("TopologicPy Version:", Helper.Version(check=True))
print("\nDONE")

## Create a sample graph

In [ ]:
c = CellComplex.Prism(uSides=3, vSides=3, wSides=3)
sample_graph = Graph.ByTopology(c, toExteriorTopologies=False)
# Set a graph_id
d = Dictionary.ByKeysValues(["graph_id", "title"], [1, "Sample Graph"])
sample_graph = Topology.SetDictionary(sample_graph, d)
vertices = Graph.Vertices(sample_graph)
for i, v in enumerate(vertices):
    d = Dictionary.ByKeyValue("label", str(i+1))
    v = Topology.SetDictionary(v, d)
Topology.Show(sample_graph, vertexSize=12, showVertexLabel=True, vertexLabelKey="label")
print("\nDone")

## Create an Induced Subgraph from 10 of the vertices

In [ ]:
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge

vertices = Graph.Vertices(sample_graph)[:10]
induced_g = Graph.InducedSubgraph(sample_graph, vertices, strict=True)
Topology.Show(induced_g, vertexSize=12, showVertexLabel=True, vertexLabelKey="label")


## Create a K-Hops Subgraph of the Sample Graph

In [ ]:
vertices = Graph.Vertices(sample_graph)[0:1]
sub_graph = Graph.KHopsSubgraph(sample_graph, vertices, k=2)
# Set a graph_id
d = Dictionary.ByKeysValues(["graph_id", "title"], [2, "Sub Graph"])
sub_graph = Topology.SetDictionary(sub_graph, d)
Topology.Show(sub_graph, vertexSize=12, showVertexLabel=True, vertexLabelKey="label")

## Specify the Kuzu database file path (usually the name ends with .kuzu) and initialize the Database manager.

In [ ]:
# Your database will be created and maintained at the following db_path
db_path = r"C:\Users\sarwj\OneDrive - Cardiff University\Desktop\Test01.kuzu"
manager = Kuzu.Manager(db_path)
print("\nDone")

## Empty the database (especially if you are repeatedly testing this workflow)

In [ ]:
status = Kuzu.EmptyDatabase(manager)
if status:
    print("The Kuzu database has been successfully emptied.")
else:
    print("Emptying the Kuzu database has failed.")
print("\nDone")

## Upsert the sample topologic graph and subgraph to the Kuzu database
(Upsert is a portmanteau of "update" and "insert".  Upsert is a database operation that inserts a new record into a table if a record with a matching key doesn't exist, or updates an existing record if a match is found)

In [ ]:
graph_id1 = Kuzu.UpsertGraph(manager, sample_graph, graphIDKey="graph_id")
print("Graph ID:", graph_id1, "has been upserted to the Kuzu database")
graph_id2 = Kuzu.UpsertGraph(manager, sub_graph, graphIDKey="graph_id")
print("Graph ID:", graph_id2, "has been upserted to the Kuzu database")
print("\nDone")

## List all current Graph Cards

In [ ]:
cards = Kuzu.ListGraphs(manager)
print("The Kuzu database contains:", len(cards), "graphs.")
for card in cards:
    print(card)

## Retrieve the graphs back through their id.

In [ ]:
graph_ids = [card['id'] for card in cards]
print("Graph IDs:", graph_ids)
new_graphs = []
for graph_id in graph_ids:
    new_graph = Kuzu.GraphByID(manager, graph_id)
    new_graphs.append(new_graph)
    Topology.Show(new_graph, vertexSize=12, showVertexLabel=True, vertexLabelKey="label")
print("\nDone")

## Check the graph dictionary

In [ ]:
for new_graph in new_graphs:
    g_dict = Topology.Dictionary(new_graph)
    print(Dictionary.Keys(g_dict), Dictionary.Values(g_dict))
print("\nDone")

## Check the vertices dictionaries of the Subgraph

In [ ]:
vertices = Graph.Vertices(new_graphs[1], sortBy=None)
for v in vertices:
    v_dict = Topology.Dictionary(v)
    print(Dictionary.Keys(v_dict), Dictionary.Values(v_dict))
print("\nDone")

## Check the edge dictionaries of the subgraph

In [ ]:
edges = Graph.Edges(new_graphs[1], sortBy="label", reverse=True)
for e in edges:
    e_dict = Topology.Dictionary(e)
    print(Dictionary.Keys(e_dict), Dictionary.Values(e_dict))
print("\nDone")

## Try ByCSV Files
You can load topologic graphs (created for machine learning) that are saved in CSV format. See Graph.ByCSVPath. Try Kuzu.ByCSVPath on your own

In [ ]:
csv_path = "INPUT YOUR CSV FOLDER PATH HERE"

status = Kuzu.EmptyDatabase(manager)
if status:
    print("The Kuzu database has been successfully emptied.")
else:
    print("Emptying the Kuzu database has failed.")
print("\nDone")

_ = Kuzu.ByCSVPath(manager, csv_path)
cards = Kuzu.ListGraphs(manager)
print("The Kuzu database contains:", len(cards), "graphs.")
print("Printing up to 10 cards or the total number of cards whichever is less.")
for card in min(len(cards), 10):
    print(card)